# Graduates Admission - Experiment Tracking

In this notebooks, we will perform following things:
1. Use PyCaret to automate model building process
1. Use MLFlow to track these experiments

## Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


## Read data

In [ ]:
raw_data = pd.read_csv('data/admission_data-v2.csv')
raw_data.head()


## MLFlow

#### Configure MLFlow URI

In [ ]:
import mlflow

mlflow.set_tracking_uri('http://localhost:5000')

experiment_name = 'Graduate Admission Predictor'
mlflow.set_experiment(experiment_name)


## PyCaret

#### Setup

In [ ]:
from pycaret.regression import *

exp_ppt101 = setup(data=raw_data, target='Chance of Admit',
                   normalize=True, normalize_method='minmax',
                   numeric_features=['University Rating'],
                   train_size=0.7, session_id=123, n_jobs=None,
                   log_experiment=True, experiment_name=experiment_name,
                   log_plots=True, log_profile=True, log_data=True
                  )


In [ ]:
get_config('X_test').head(5)


#### Compare various models

In [ ]:
top3 = compare_models(n_select=3)


#### Hyperparameter tuning

In [ ]:
tuned_models = []

for m in top3:
    tuned_models.append(tune_model(m))
    

In [ ]:
tuned_models


## Model as Python Function

In [ ]:
model_uri = ''

loaded_model = mlflow.pyfunc.load_model(model_uri)

production_data = pd.read_csv('./data/admission_data-production.csv')

loaded_model.predict(production_data)


## Model from Model Registry

The final model can be moved to Model Registry to declare the production grade model published for the given use case.

In [ ]:
model_name = 'Graduate Admission Predictor'
stage = 'production'

# Load model
model_uri = f'models:/{model_name}/{stage}'
model_from_registry = mlflow.sklearn.load_model(model_uri)

# Load data
production_data = pd.read_csv('./data/admission_data-production.csv')

# Predict
model_from_registry.predict(production_data)


## Model as Endpoint

In [ ]:
import requests

model_uri = 'http://0.0.0.0:5001'

production_data = pd.read_csv('./data/admission_data-production.csv')

def predict(data, _model_uri):
    headers = {
        "Content-Type": "application/json",
      }
    data_json = data.to_dict(orient='records')
    dataframe_records = {'dataframe_records': data_json}
    response = requests.request(method='POST', headers=headers, url=f'{_model_uri}/invocations', json=dataframe_records)
    if response.status_code != 200:
      raise Exception(f"Request failed with status {response.status_code}, {response.text}")
    return response.json()

predict(production_data, model_uri)
